# Goldman equation

The transmembrane potential of a semipermeable membrane is dependent on the concentration [X] and permitivity $g_X$ of ions.
When more than one ion species is considered around a semipermeable membrane the equilibrium potential is given by the Goldman eq, for Sodium and Potassium we can write this as

$$
V_m ={\frac  {RT}{F}}\ln {\left({\frac  {g_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {out}}}+g_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{\mathrm  {out}}}}{g_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {in}}}+g_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{{\mathrm  {in}}}}}}\right)}
$$
$$
%V_m ={\frac  {RT}{F}}\ln {\left({\frac  {P_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {out}}}+P_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{\mathrm  {out}}}+P_{{{\text{Cl}}}}[{\text{Cl}}^{{-}}]_{{\mathrm  {in}}}}{P_{{{\text{Na}}}}[{\text{Na}}^{{+}}]_{{\mathrm  {in}}}+P_{{{\text{K}}}}[{\text{K}}^{{+}}]_{{{\mathrm  {in}}}}+P_{{{\text{Cl}}}}[{\text{Cl}}^{{-}}]_{{\mathrm  {out}}}}}\right)}
$$

In the following section, this equation is plotted as a function of some concentration [X]_{somewhere} and conductiviti $g_X$, so you get an idea what this equation does.

## Effect of extracellular $[K^+]_\text{out}$ 
- How does the membrane equilibrium potential change when the extracellular Poassium Concentration is changed?

First, the Goldman eq. is defined in python code.
- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def vm_goldmann(g_Na=0.05, g_K=1.0,
                Na_in=15, Na_out=145, 
                K_in=140, K_out=4):
    """The Goldman-Hodgkin-Katz equation.
       input: Conductivities and concentrations in arbitrary units.
       returns: Vm in volt."""
    
    # RT/F at room temperature
    prefactor = 26.7e-3 # Volt
    
    # the conductivity weighted concentration ratio sum
    gc_ratio = (g_Na*Na_out + g_K * K_out)/(g_Na*Na_in + g_K * K_in)
    
    # Return the transmembrane potential
    vm = prefactor * np.log(gc_ratio)
    return vm

Then we can plot $V_m([K^+]_\text{out})$. We present three cases:
1. only $g_Na$ 
2. only $g_K$ 
3. $g_K = 20 g_Na$ 

- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["font.size"] = 22
#matplotlib.rcParams.update({'font.size': 22})

K_outs = np.logspace(-1,4)
plt.plot(K_outs, 1e3*vm_goldmann(K_out=K_outs), label="Standard")
plt.plot(K_outs, 1e3*vm_goldmann(K_out=K_outs, g_Na=0), label="No Na channels")
plt.plot(K_outs, 1e3*vm_goldmann(K_out=K_outs, g_K=0), label="No K channels")
plt.semilogx()
plt.ylabel("$V_m (mV)$")
plt.xlabel("K_out")
plt.legend();

## Goldmann: g ratio

- How does the membrane equilibrium potential change when the ratio $g_{Na}/g_k$ is changed?

- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER

In [ ]:
g_Nas = np.logspace(-4,4)
plt.plot(g_Nas, 1e3*vm_goldmann(g_Na=g_Nas), label="4 (standard)")
plt.plot(g_Nas, 1e3*vm_goldmann(g_Na=g_Nas, K_out=10), label="10")
plt.plot(g_Nas, 1e3*vm_goldmann(g_Na=g_Nas, K_out=140), label="140 ( = [K]$_{\mathrm{in}}$ )")
plt.legend(title="[K]$_{\mathrm{out}}$")
plt.semilogx()
plt.ylabel("$V_m (mV)$")
plt.xlabel("$g_{Na} / g_{K}$") # Since g_k is 1

# Current injection at passive Cable

What happens when current is injected in a thin nerve fiber, how does the current spread?


The answer is given by the core=conductor- or cable-model. 
It is a very general model that can be dated back to Lord Kelvin who derived it to describe transatlatic communication cables. Bioelectricity, e.g. in nerve fibers, obeys the same physical laws. 
More generally speaking the cable equation is the one dimensional heat equation. So you can think of a thin metal rod that is heated at one point (How does the heat spread?).

In this model the ion species are not distinguished, there is just one current (of course still with a capacitive and a resistive component). We use a library called Brian.
- The capacitive current is calculated inside of Brian. You just define the specific capacity Cm, and the geometry.
- The resistive current is given by $$ I_m = g_L * (E_L - V_m) $$ 

The following code inject current at one end of a finite (not infinite) passive fiber. 
You may change capacity, Radius, current injection amplitude, leakynes .... 

- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER

(In case this Notebook has problems, A similar example code can be found and run interactively here:
https://hub.mybinder.turing.ac.uk/user/brian-team-brian2-binder-dymv2fxx/notebooks/examples/compartmental/cylinder.ipynb. You can copy paste the code below.)


In [ ]:
from brian2 import *

defaultclock.dt = 0.01*ms

# Morphology
diameter = 1*um
length = 300*um
Cm = 1*uF/cm**2
Ri = 150*ohm*cm
N = 200
morpho = Cylinder(diameter=diameter, length=length, n=N)

# Passive channels
gL = 1e-4*siemens/cm**2
EL = -70*mV
eqs = '''
Im = gL * (EL - v) : amp/meter**2
I : amp (point current)
'''

neuron = SpatialNeuron(morphology=morpho, model=eqs, Cm=Cm, Ri=Ri,
                       method='exponential_euler')
neuron.v = EL
M = StateMonitor(neuron, 'v', record=[0, 10,100,199])

la = neuron.space_constant[0]
print("Electrotonic length: %s" % la.in_best_unit(2))

neuron.I[0] = 0.02*nA # injecting at the left end

run(.1*ms, report='text')
plot(neuron.distance/um, neuron.v/mV, 'k:', label="0.1 ms")
run(0.9*ms, report='text')
plot(neuron.distance/um, neuron.v/mV, 'k--', label="1.0 ms")
run(9*ms, report='text')
plot(neuron.distance/um, neuron.v/mV, 'k+', label="10.0 ms")

# Theory
x = neuron.distance
ra = la * 4 * Ri / (pi * diameter**2)
theory = EL + ra * neuron.I[0] * cosh((length - x) / la) / sinh(length / la)
plot(x/um, theory/mV, 'r', label=r"Theory $t\rightarrow\infty$")
xlabel('x (um)')
ylabel('v (mV)')
legend(title="time")

figure()
plot(M.t/ms, M.v.T/mV, label= x[M.record])
legend(title="Distance from injection", loc="best", ncol=2)
xlabel('t (ms)')
ylabel('v (mV)')

# HH Action Potential

An AP is a rapid and propagating jump in the transmembrane potential. There are many models to describe such a process.
The most famous one is the HH model, which is a very detailed purely electrical model.
In the first Section it was found that a change in ion specific permeabilities effects the membrane potential. 

The HH model uses the Goldman eq. and describes the permeabilities with 
voltage and time dependent expressions.
It has more than 20 constant parameters and 3 internal variables (m, n, h).
With so many parameters, various AP shapes can be reproduced.
However, these properties can not be monitored or controlled so easily, and it is not reaally clear what m, n and h represent in an actual neuron.


The following code simuates a HH AP along a long axon, and calculates the propagation velocity.

- Run the following code by selecting (click on it) and then pressing SHIFT+ENTER or CTRL+ENTER
  (If it doesnt work you can also try at https://mybinder.org/v2/gh/brian-team/brian2-binder/master?filepath=examples/compartmental/hh_with_spikes.ipynb)
- whats the AP velocity?

The nerve is almost half a millimeter thick and still the AP speed is rather disenchanting (should be around 12 m/s)
You have heard of myelin which wraps around axons and increases the AP propagation speed.
We can include the insulating effect of myelin by decreasing the leak condictivity $g_L$.
(Note, divide $g_L$ by 2 would double the so called Electrotonic length or "length parameter")

- Try to make the AP double its speed by decresing $g_L$.
  What is the relation between length parameter and propagation speed?

- Change $C_m$ to reach the desired velocity.

- Bonus for fun: Are there other ways to accelerate the AP?

In [ ]:
from scipy import stats

start_scope()
defaultclock.dt = 0.01*ms

morpho = Cylinder(length=10*cm, diameter=2*238*um, n=1000, type='axon')

El = 10.613*mV
ENa = 115*mV
EK = -12*mV
gl = 0.4*msiemens/cm**2
gNa0 = 120*msiemens/cm**2
gK = 36*msiemens/cm**2

# HH equations
eqs = '''
# The same equations for the whole neuron, but possibly different parameter values
# distributed transmembrane current
Im = gl * (El-v) + gNa * m**3 * h * (ENa-v) + gK * n**4 * (EK-v) : amp/meter**2
I : amp (point current) # applied current
dm/dt = alpham * (1-m) - betam * m : 1
dn/dt = alphan * (1-n) - betan * n : 1
dh/dt = alphah * (1-h) - betah * h : 1
alpham = (0.1/mV) * 10.0*mV/exprel((-v+25.0*mV)/(10.0*mV))/ms : Hz
betam = 4 * exp(-v/(18.0*mV))/ms : Hz
alphah = 0.07 * exp(-v/(20.0*mV))/ms : Hz
betah = 1/(exp((-v+30.0*mV) / (10.0*mV)) + 1)/ms : Hz
alphan = (0.01/mV) * 10.0*mV/exprel((-v+10.0*mV)/(10.0*mV))/ms : Hz
betan = 0.125*exp(-v/(80.0*mV))/ms : Hz
gNa : siemens/meter**2
'''

neuron = SpatialNeuron(morphology=morpho, model=eqs, method="exponential_euler",
                       refractory="m > 0.4", threshold="m > 0.5",
                       Cm=1*uF/cm**2, Ri=35.4*ohm*cm)
neuron.v = 0*mV
neuron.h = 1
neuron.m = 0
neuron.n = .5
neuron.I = 0*amp
neuron.gNa = gNa0
M = StateMonitor(neuron, 'v', record=True)
spikes = SpikeMonitor(neuron)

run(50*ms, report='text')
neuron.I[0] = 1*uA # current injection at one end
run(3*ms)
neuron.I = 0*amp
run(50*ms, report='text')

In [ ]:
# Calculation of velocity
slope, intercept, r_value, p_value, std_err = stats.linregress(spikes.t/second,
                                                neuron.distance[spikes.i]/meter)
print("Velocity = %.2f m/s" % slope)

subplot(211)
for i in range(10):
    plot(M.t/ms, M.v.T[:, i*100]/mV)
ylabel('v')
subplot(212)
plot(spikes.t/ms, spikes.i*neuron.length[0]/cm, '.k')
plot(spikes.t/ms, (intercept+slope*(spikes.t/second))/cm, 'r')
xlabel('Time (ms)')
ylabel('Position (cm)')
show()